In [2]:
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

True

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [4]:
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    max_retries=2,
)

In [15]:
result=llm.invoke('What is two sum problem? Explain in 50 word or less.')

In [17]:
result.content

"The Two Sum problem asks you to find two numbers in a given array of integers that sum up to a specific target number. You typically return the indices of these two numbers. It's a classic problem, often solved efficiently using a hash map."

The Two Sum problem asks you to find two numbers in a given array of integers that sum up to a specific target number. You typically return the indices of these two numbers. It's a classic problem, often solved efficiently using a hash map.

In [21]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

In [44]:
python_repl = PythonREPL()

In [45]:
python_repl.run("print(1+1)")

'2\n'

In [47]:
python_repl.run("""
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

fib(10)
""")


'NameError("name \'fib\' is not defined")'

In [37]:
from langchain_experimental.utilities import PythonREPL

class PersistentPythonREPLTool(PythonREPL):
    def __init__(self):
        super().__init__()
        self._globals = {}

    def run(self, command: str) -> str:
        try:
            # First try eval (single expression)
            result = eval(command, self._globals)
            return str(result)
        except SyntaxError:
            # If it's a statement block, use exec
            try:
                exec(command, self._globals)
                return "Executed."
            except Exception as e:
                return repr(e)
        except Exception as e:
            return repr(e)


In [38]:
python_repl = PersistentPythonREPLTool()

# Define and use the function in one call
print(python_repl.run("""
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)
"""))

# Call the function later
print(python_repl.run("fib(10)"))


Executed.
55


In [48]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be" \
                " a valid python command. If you want to see the output of a value, you " \
                "should print it out with `print(...)`.",
    func=python_repl.run,
)

In [50]:
llm_with_tools=llm.bind_tools([repl_tool])

In [52]:
result=llm_with_tools.invoke("What is two sum problem? Explain in 50 word or less. and run the code with the tool to show me some test cases")

In [53]:
result

AIMessage(content='The Two Sum problem asks you to find two numbers in an array that add up to a specific target. You need to return the indices of these two numbers.\n\nHere are some test cases:', additional_kwargs={'function_call': {'name': 'python_repl', 'arguments': '{"__arg1": "\\ndef two_sum(nums, target):\\n    num_map = {}\\n    for i, num in enumerate(nums):\\n        complement = target - num\\n        if complement in num_map:\\n            return [num_map[complement], i]\\n        num_map[num] = i\\n    return []\\n\\nprint(two_sum([2, 7, 11, 15], 9))\\nprint(two_sum([3, 2, 4], 6))\\nprint(two_sum([3, 3], 6))\\nprint(two_sum([1, 2, 3, 4, 5], 10))\\n"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb69a9d0-4c52-4f58-916f-92ed58ba8b82-0', tool_calls=[{'name': 'python_repl', 'args': {'__arg1': '\ndef two_sum(nums, target):\n    num_map = {}\n    for

AIMessage(content='The Two Sum problem asks you to find two numbers in an array that add up to a specific target. You need to return the indices of these two numbers.
\n\nHere are some test cases:',

 additional_kwargs={'function_call': {'name': 'python_repl', 'arguments': '{"__arg1": 
 
 "\\ndef two_sum(nums, target):\\n    num_map = {}\\n    for i, num in enumerate(nums):\\n        complement = target - num\\n        if complement in num_map:\\n            return [num_map[complement], i]\\n        num_map[num] = i\\n    return []\\n\\nprint(two_sum([2, 7, 11, 15], 9))\\nprint(two_sum([3, 2, 4], 6))\\nprint(two_sum([3, 3], 6))\\nprint(two_sum([1, 2, 3, 4, 5], 10))\\n"}'}},
 
 
  response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb69a9d0-4c52-4f58-916f-92ed58ba8b82-0',
  
   tool_calls=[{'name': 'python_repl', 'args': {'__arg1': '\ndef two_sum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n    return []\n\nprint(two_sum([2, 7, 11, 15], 9))\nprint(two_sum([3, 2, 4], 6))\nprint(two_sum([3, 3], 6))\nprint(two_sum([1, 2, 3, 4, 5], 10))\n'}, 
   
   
   'id': 'fca36480-fd74-4c35-a973-8f0ecc450a87', 'type': 'tool_call'}], usage_metadata={'input_tokens': 105, 'output_tokens': 311, 'total_tokens': 416, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 100}})

In [54]:
result.tool_calls

[{'name': 'python_repl',
  'args': {'__arg1': '\ndef two_sum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n    return []\n\nprint(two_sum([2, 7, 11, 15], 9))\nprint(two_sum([3, 2, 4], 6))\nprint(two_sum([3, 3], 6))\nprint(two_sum([1, 2, 3, 4, 5], 10))\n'},
  'id': 'fca36480-fd74-4c35-a973-8f0ecc450a87',
  'type': 'tool_call'}]

In [86]:
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [87]:
from langchain_tavily import TavilySearch

In [140]:
from langchain_core.tools import tool

@tool(name_or_callable="tavily_search", description="Use Tavily to search the web for current or recent events.")
def tavily_search(query: str) -> str:
    search = TavilySearch(  
        max_results=5,
        topic="general",
        )
    response = search.invoke({"query": query})
    # Return a summary of top result(s) — you could customize this
    results = response.get("results", [])
    if not results:
        return "No results found."
    return results[0].get("content", "No content available.")

In [169]:
def generate_hint(question: str) -> str:
    """Generate a helpful hint for a DSA problem without solving it.
    input: question (str): The DSA problem to generate a hint for.
    output: str: A helpful hint for the DSA problem."""
    
    return llm.invoke(f"Give a helpful hint for this DSA problem without solving it: {question}")


In [170]:
def generate_test_cases(problem_description: str) -> str:
    """Use this to generate test cases for DSA problems.
    input: problem_description - the DSA problem statement
    output: 3 test cases without solving the problem"""
    
    return llm.invoke(f"Create 3 test cases for this DSA problem without solving it: {problem_description}")


In [171]:
def bug_hint_tool(code: str) -> str:
    """Use this to analyze code for logic issues and provide a subtle hint.
    input: code - the code to analyze
    output: str - a subtle hint about potential logic issues in the code.
    """

    return llm.invoke(f"Analyze this code for logic issues and give a subtle hint: {code}")


In [172]:
llm_with_tools = llm.bind_tools([repl_tool, tavily_search,generate_hint, generate_test_cases, bug_hint_tool])

In [173]:
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, END

In [174]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [175]:
from typing import Callable

# You must register your tool functions in a dictionary
tool_map: dict[str, Callable] = {
    "python_repl": python_repl.run,
    "search_tool": tavily_search,
    "generate_hint": generate_hint,
    "generate_test_cases": generate_test_cases,
    "bug_hint_tool": bug_hint_tool,
}

In [178]:
def execute_tools(state: State) -> State:
    last_message = state["messages"][-1]
    tool_outputs = []

    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        # Get function from registered tool map
        tool_func = tool_map.get(tool_name)

        if tool_func is None:
            output = f"[ERROR] Unknown tool: {tool_name}"
        else:
            try:
                # Prefer full argument dict if function accepts it
                output = tool_func(tool_args)
            except Exception as e:
                output = f"[ERROR while running {tool_name}]: {e}"

        tool_outputs.append(
            ToolMessage(
                tool_call_id=tool_call["id"],
                content=output
            )
        )

    return {"messages": state["messages"] + tool_outputs}


In [179]:
def should_call_tool(state: State) -> str:
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "call_tool"
    return END

In [180]:
def call_llm(state: State) -> State:
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [78]:
graph = StateGraph(State)

graph.add_node("call_llm", call_llm)
graph.add_node("call_tool", execute_tools)
graph.set_entry_point("call_llm")
graph.add_conditional_edges("call_llm", should_call_tool)
graph.add_edge("call_tool", "call_llm")

app = graph.compile()


In [142]:
inputs = {
    "messages": [HumanMessage(content="What is two sum problem? Show test cases using Python and run it show output with proper debugging.")]
}

final_state = app.invoke(inputs)


In [143]:
for message in final_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is two sum problem? Show test cases using Python.
================================== Ai Message ==================================

The Two Sum problem is a classic algorithmic challenge. Given an array of integers `nums` and an integer `target`, the goal is to find the indices of two numbers in the array that add up to the `target`. You can assume that each input will have exactly one solution, and you may not use the same element twice.

Here's an example implementation and test cases in Python:
Tool Calls:
  python_repl (8dc0428c-6dd6-495c-ac57-ad20849f9ecb)
 Call ID: 8dc0428c-6dd6-495c-ac57-ad20849f9ecb
  Args:
    __arg1: 
def two_sum(nums, target):
    num_map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    return []

# Test Cases
print(f"Test Case 1: nums=[2, 7, 

In [148]:
def call_llm(state: State) -> State:
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}